<a href="https://colab.research.google.com/github/sahaanirbannew/spacy-custom-NER-bird-names/blob/main/Program_fetch_tweet_based_on_search_term%2C_create_training_data%2C_ML_model_to_NER_bird_names.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports & Function definitions

In [ ]:
!pip install ekphrasis

In [24]:
def what_search_term():
  search_term = "indiAves"        ## CHANGE THIS. 
  #search_term = "birdphotography"
  return search_term
  

import tweepy
import time
import pickle
import csv
import datetime 
from datetime import datetime 
import os, sys
from google.colab import drive
import pandas as pd 
import re
from spacy.tokens import DocBin
from tqdm import tqdm
from spacy.util import filter_spans
import shutil
from zipfile import ZipFile
import tensorflow as tf
from ekphrasis.classes.segmenter import Segmenter
seg_tw = Segmenter(corpus="twitter")

def get_common_name_list():
  filepath = open("/content/dict_commName_sciName",'rb') 
  common_name_list = pickle.load(filepath)
  filepath.close()  
  return common_name_list

def connect_to_google_drive():
  drive.mount("/content/drive", force_remount=False)

def extract_model_best_from_archive():
  file_name = "/content/model-best.zip"
  with ZipFile(file_name, 'r') as zip:
    zip.extractall("/content/model-best")

def is_GPU_on():
  if tf.test.gpu_device_name() == "": 
    return False
  return True 

def create_twitter_app_obj():
  consumer_key = "iPaIdR8GRI59yTJMs0Es0dIBN"
  consumer_secret = "pLadg3UaLeK3yKDujRMChRN3p8hUDBOjBsuOBy8j8ERr4zz1vs"
  access_token = "39085479-AabHt6bmFSbClDfUZuHjModYPAxVlOxHeMA79UyVt"
  access_token_secret = "3IqXDISfqg14wzMNNn2AX4KYG9Wfkltt21QxKasE4YNnG"
  # Creating the authentication object
  auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
  # Setting your access token and secret
  auth.set_access_token(access_token, access_token_secret)
  # Creating the API object while passing in auth information

  try:
    api = tweepy.API(auth) 
    return api
  except:
    print("Error: Error making the Twitter Object.")
    return 0

def get_google_drive_folder_path():
  path = "/content/drive/My Drive/IndiAves/" 
  return path 

def get_since_id(path,search_word):
  file = open(path+"since_id",'rb')
  try:
    since_id_data = pickle.load(file)
  except Exception as e:
    print(str(e))
    return 0
  try: 
    since_id = since_id_data[search_word]
    return since_id
  except:
    print("Info: This looks like a new search.")
    return 0

def get_eBird_commonNames_data(path):
  file = open(path+"bird_dict_comName",'rb')
  try:
    eBird_commonNames_data = pickle.load(file)
    return eBird_commonNames_data 
  except Exception as e:
    print(str(e))
    return 0

def load_all_birds_list(path):
  file = open(path+"bird_list_df",'rb')
  bird_list_df = pickle.load(file)
  try: 
    return bird_list_df
  except:
    print("Error: No bird list found.")
    return 0
  
def load_training_data(path):
  file = open(path+"training_data",'rb')
  training_data = pickle.load(file)
  try: 
    return training_data
  except:
    print("Error: No training data found.")
    return 0

def load_nlp_model_():
  import spacy
  #nlp = spacy.load("en_core_web_sm")
  nlp = spacy.blank("en")
  print(nlp.pipe_names)
  return nlp

def create_doc_bin(training_data,nlp):
  from spacy.tokens import DocBin
  from tqdm import tqdm
  from spacy.util import filter_spans
  doc_bin = DocBin()
  for training_example  in tqdm(training_data['annotations']): 
    text = training_example['text']
    labels = training_example['entities']
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in labels:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        #if span is None:
        #    print("Skipping entity")
        #else:
        #    ents.append(span)
        if span is not None:
          ents.append(span)
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents 
    doc_bin.add(doc)
  doc_bin.to_disk("training_data.spacy") # save the docbin object

def export_training_data(training_data,path):
  filepath = open(path+"training_data",'wb') 
  pickle.dump(training_data, filepath)                     
  filepath.close() 
  print(len(training_data["annotations"]), "records in training data.")
  print("training data is saved.") 


def initialisation():
  !pip install tweet-preprocessor
  !python -m spacy download en_core_web_sm 
  !pip install spacy[transformers]
  !python -m spacy download en_core_web_lg 
  !python -m spacy download en_core_web_trf
  !pip install ekphrasis

def import_base_config_file(google_drive_folder_path):
  shutil.copyfile(google_drive_folder_path+"base_config.cfg", "/content/base_config.cfg")

spelling_corrections = {}
spelling_corrections["grey"] = "gray" 
spelling_corrections["pegion"] = "pigeon" 
spelling_corrections["brested"] = "breasted" 
spelling_corrections["serpant"] = "serpent" 
spelling_corrections["avedavat"] = "avadavat" 
spelling_corrections["open billed stork"] = "asian openbill" 
spelling_corrections["secretary bird"] = "Secretarybird" 
spelling_corrections["dollar bird"] = "dollarbird"

def preprocess_tweets_arr_batch(tweets,spelling_corrections,all_birds):
  new_tweets_ = []
  for tweet in tweets:
    tweet = basic_preprocess(tweet,spelling_corrections, all_birds)
    if len(tweet) > 1:
      new_tweets_.append(tweet)
  return new_tweets_

def add_to_database(api, search_word, ext_path, since_id):
  new_search = "#" +search_word + " -filter:retweets" 
  count = 0
  csvFile = open(ext_path+search_word+'.csv', 'a')
  csvFile2 = open('/content/temp_.csv', 'w') 

  csvWriter = csv.writer(csvFile)
  csvWriter2 = csv.writer(csvFile2)

  for tweet in tweepy.Cursor(api.search,q=new_search,count=100,
                            lang="en",
                            since_id=since_id, tweet_mode="extended").items(): 
    if tweet.id > since_id:
      since_id = tweet.id
    
    media_url=""
    if tweet.entities.get('media', []): media_url = tweet.entities.get('media', [])[0]['media_url']

    hashtags = []
    for tag in tweet.entities["hashtags"]:
      hashtags.append(tag["text"]) 
    
    csvWriter.writerow([tweet.created_at, tweet.id, tweet.user.screen_name.encode('utf-8'), tweet.user.location.encode('utf-8'), tweet.full_text.encode('utf-8'), media_url, hashtags])
    csvWriter2.writerow([tweet.created_at, tweet.id, tweet.user.screen_name.encode('utf-8'), tweet.user.location.encode('utf-8'), tweet.full_text.encode('utf-8'), media_url, hashtags])
    count +=1
  print(datetime.now(),count, "tweets retrieved.")
  return since_id 

def update_since_id(since_id, path,search_word):
  filepath = open(path+"since_id",'rb')
  since_id_data = pickle.load(filepath)
  since_id_data[search_word]=since_id
  filepath = open(path+"since_id",'wb') 
  pickle.dump(since_id_data, filepath)                     
  filepath.close()

def train_model():
  !python -m spacy init fill-config base_config.cfg config.cfg
  !python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy --gpu-id 0

def import_best_model(google_drive_folder_path):
  shutil.copyfile(google_drive_folder_path+"model-best.zip", "/content/model-best.zip")

def archive_best_model_in_google_drive(google_folder_path):
  shutil.make_archive(google_folder_path+"model-best", 'zip', "/content/model-best")

def train_and_load_best_model(training_data,nlp,google_drive_folder_path): 
  is_nlp_ready = True 

  try:
    is_GPU_on_colab = is_GPU_on() 
    print("is_GPU_on",is_GPU_on_colab)
    if is_GPU_on_colab == True: 
      create_doc_bin(training_data,nlp)

      #train the model 
      try:
        import_base_config_file(google_drive_folder_path)
        print("Base Config File fetched.")
      except Exception as e: 
        print(str(e))
        print("Error: Base Config File is NOT fetched.")

      #trains the model.
      try:
        train_model()
        #archive the best model to Google Drive. 
        archive_best_model_in_google_drive(google_drive_folder_path)
      except Exception as e:
        print(str(e))
        print("Error: Model training failed.")
        is_GPU_on_colab = False #so that it fetches the last saved model. 
    
    if is_GPU_on_colab == False: 
      import_best_model(google_drive_folder_path)
      extract_model_best_from_archive()
    
    #load the best model
    nlp_ner = spacy.load("model-best") 
  except Exception as e:
    return False, ""

  return is_nlp_ready, nlp_ner


def load_best_model_from_archive(google_drive_folder_path):
  import_best_model(google_drive_folder_path)
  extract_model_best_from_archive()
  nlp_ner = spacy.load("model-best") 
  return True, nlp_ner

def visualize_entities(pipeline, text): 
    document = pipeline(text) 
    displacy.render(document, jupyter=True, style='ent')

import requests 
def update_training_data(training_data,new_tweets,all_birds_name,is_nlp_ready,nlp_ner, ebird_names):
  commNames_wb = get_common_name_list() 
  for tweet in new_tweets:
    tweet_bird_found = False 
    print("------------") 
    print(tweet) 
    api_url = "https://bird-name-ner-nlp.herokuapp.com/ner?sent="+tweet
    response = requests.get(api_url).json()
    print("bird-ner: ", response['bird-ner']) 
    print("bird-wiki: ",response['bird-wiki']) 
    print("bird-ebird: ",response['bird-ebird']) 
    visualize_entities(location_nlp, tweet)


    
    bird_list_from_tweet = [] 
    for bird in response['bird-wiki']: 
      bird = bird.strip() 
      if bird not in bird_list_from_tweet: 
        bird_list_from_tweet.append(bird)
    for bird in response['bird-ebird']: 
      bird = bird.strip()
      if bird not in bird_list_from_tweet: 
        bird_list_from_tweet.append(bird)

    if len(response['bird-wiki'])>0 and len(response['bird-ner'])>0 and len(response['bird-ebird'])>0:
      if response['bird-wiki'] == response['bird-ner'] and response['bird-wiki'] == response['bird-ebird']:
        print("Wiki & eBird have predicted the same. No dispute!") 

    if len(response['bird-wiki'])>0 and len(response['bird-ebird'])>0:
      if response['bird-wiki'] == response['bird-ebird']:
        print("All three have predicted the same. No dispute!")

    print("--Suggestion from NER model: "+str(response['bird-ner']))
    if len(response['bird-wiki']) == 0 and response['bird-ebird'] == 0: 
      for bird in response['bird-ner']: 
        if bird in commNames_wb: 
          print(bird, " exists. It is a common name.")

    birds_input = input("Birds,:").split(",")
    if birds_input != "-":
      for bird_ in birds_input:  
        bird_ = bird_.strip()
        if bird_ not in bird_list_from_tweet: 
          bird_list_from_tweet.append(bird_) 

    # Now bird_list_from_tweet has a complete list of birds found in the tweet. 
    # Make the training data. 
    for bird in bird_list_from_tweet: 
      print(bird, tweet.find(bird), tweet.find(bird)+len(bird))
      training_data["annotations"].append({'text': tweet, 'entities': [(tweet.find(bird), tweet.find(bird)+len(bird), 'BIRDNAME')]}) 
  return training_data 

def return_eBird_list(spelling_corrections, all_birds):
  eBird_commonNames = get_eBird_commonNames_data(google_drive_folder_path)
  eBird_commonNames_list = []
  for eBird in eBird_commonNames:
    if eBird_commonNames[eBird] != "ou":
      eBird_commonNames[eBird] = basic_preprocess(eBird_commonNames[eBird],spelling_corrections, all_birds)
      eBird_commonNames_list.append(eBird_commonNames[eBird].strip())
  return eBird_commonNames_list 

def basic_preprocess(tweet,spelling_corrections, all_birds):
  import preprocessor as p
  p.set_options(p.OPT.EMOJI, p.OPT.MENTION, p.OPT.URL, p.OPT.SMILEY, p.OPT.NUMBER, p.OPT.HASHTAG)
  
  tweet = tweet.lower()

  hashtags = re.findall(r"#(\w+)", tweet)  
  for hashtag in hashtags:
    segmented_ = seg_tw.segment(hashtag) 
    for bird in all_birds: 
      if bird.find(segmented_) > -1: 
        tweet = tweet.replace("#"+hashtag,seg_tw.segment(hashtag))
        break

  tweet = tweet.replace("\n"," ")  
  tweet = tweet.replace("\\n"," ")
  tweet = p.clean(tweet) 
  if tweet[:2] == "b'": tweet = tweet[1:] 
  tweet = tweet.replace("'","")
  tweet = tweet.replace("#","")
  tweet = re.sub(r'[^\w\s]', ' ', tweet)
  tweet = re.sub(r' x..', '', tweet)
  tweet = re.sub(r' +', ' ', tweet) #' +', ' '
  #tweet = re.sub(r' n. ', '', tweet) 
  tweet = tweet.replace("x9c","")
  tweet = tweet.strip()

  for key in spelling_corrections: 
    if tweet.find(key)>-1: 
      tweet = tweet.replace(key,spelling_corrections[key])
  return tweet



Reading twitter - 1grams ...
Reading twitter - 2grams ...


# Main Program - 1

Run this only once.

Note:


*   Please use GPU to train model
*   If GPU is not available, then last best model would be loaded. 
*   Assumption: All files would be available in the folder 




In [ ]:
initialisation() 
import spacy


#what is the search term?
search_word = what_search_term() 
print(search_word)

#connect to google drive
connect_to_google_drive()

#connect to twitter api. 
twitter_api_obj = create_twitter_app_obj() 

#get Google Drive folder path
google_drive_folder_path = get_google_drive_folder_path()  

#get since_id based on the search term
since_id = get_since_id(google_drive_folder_path,search_word) 
#print(since_id)

#load all birds list 
all_birds = load_all_birds_list(google_drive_folder_path)

#load training data
training_data = load_training_data(google_drive_folder_path) 

#Set up Model Training. 
# PS: GPU needed. 
nlp = load_nlp_model_() #this is spacy stuff. 

is_nlp_ready,nlp_ner = load_best_model_from_archive(google_drive_folder_path) #for quicker work

# Main Program - 2

Keep re-running this!

In [25]:
#fetch tweets
since_id = add_to_database(twitter_api_obj,search_word, google_drive_folder_path, since_id) 
new_tweets_df = pd.read_csv("/content/temp_.csv", names=["created_at", "tweet_id", "user", "location", "tweet", "media_url", "hashtags"])
all_birds_name = all_birds["bird_name"].tolist() 
ebird_names = return_eBird_list(spelling_corrections, [])

## just making a list combining Wikipedia and the eBird lists.
new_complete_bird_list = []
for bird in all_birds_name: 
  new_complete_bird_list.append(bird)
for bird in ebird_names: 
  if bird not in new_complete_bird_list:
    new_complete_bird_list.append(bird)
new_complete_bird_list = set(new_complete_bird_list)


new_tweets = preprocess_tweets_arr_batch(new_tweets_df["tweet"].tolist(),spelling_corrections, new_complete_bird_list)


#Update training data. ###NEEDS USER INTERVENTION. 
update_training_data(training_data,new_tweets,all_birds_name,is_nlp_ready,nlp_ner, ebird_names) 

#Exports training data 
export_training_data(training_data,google_drive_folder_path)

#Update since_id
update_since_id(since_id,google_drive_folder_path,search_word)

#delete temp_.csv 
os.remove("/content/temp_.csv")

#copy the training data to baba's folder so that I can train the model.
shutil.copyfile("/content/drive/My Drive/IndiAves/training_data", "/content/drive/My Drive/IndiAves_baba/training_data")


#Train the model and save the best model.-
is_nlp_ready,nlp_ner = train_and_load_best_model(training_data,nlp,google_drive_folder_path)

2022-09-15 12:08:21.573944 0 tweets retrieved.
3865 records in training data.
training data is saved.
is_GPU_on False


# Enter singular training data, NOT from tweets.

In [ ]:
def add_singular_training_data(sentence, training_data): 
  api_url = "https://bird-name-ner-nlp.herokuapp.com/ner?sent="+sentence
  response = requests.get(api_url).json()
  print(response['bird-ner']) 
  print(response['bird-wiki']) 
  print(response['bird-ebird']) 
  birds_input = input("Birds,:").split(",")
  if birds_input != "-":
    for bird_ in birds_input: 
      bird_ = bird_.strip()
      print(bird_, sentence.find(bird_),sentence.find(bird_)+len(bird_))
      training_data["annotations"].append({'text': sentence, 'entities': [(sentence.find(bird_), sentence.find(bird_)+len(bird_), 'BIRDNAME')]})
  
  return training_data

In [ ]:
import requests
#load training data
training_data = load_training_data(google_drive_folder_path) 
training_data = add_singular_training_data(basic_preprocess(input("Enter Sentence: "),spelling_corrections), training_data)
#Exports training data 
export_training_data(training_data,google_drive_folder_path)
#copy the training data to baba's folder so that I can train the model.
shutil.copyfile("/content/drive/My Drive/IndiAves/training_data", "/content/drive/My Drive/IndiAves_baba/training_data")


Enter Sentence: grey heron
['grey heron']
['grey heron']
[]
Birds,:grey heron
grey heron 0 10
3468 records in training data.
training data is saved.


'/content/drive/My Drive/IndiAves_baba/training_data'

# Stand alone test.

In [ ]:
nlp_ner = spacy.load("model-best") 
sentence = "Periscope… no… it’s Great Egret. #IndiAves #BBCWildlifePOTD #natgeoindia #SonyAlpha #ThePhotoHour Pic taken at Bps, Mumbai. #thanecreekflamingosanctuary"
doc = nlp_ner(sentence)
print(doc.ents)

()


# Demo of the API

In [ ]:
#@title Default title text
import requests

#sentence = "b youll find them black rumped flameback spotted owlet common iora on a tree because these birds thrive in earlybirdgames indiaves birdwatching flashcardgames"
#sentence = "Periscope… no… it’s Great Egret. #IndiAves #BBCWildlifePOTD #natgeoindia #SonyAlpha #ThePhotoHour Pic taken at Bps, Mumbai. #thanecreekflamingosanctuary"
sentence = "Listen to your instincts, ignore everything The grey heron #IndiAves #TwitterNatureCommunity #NikonIndia #BBCWildlifePOTD #birdwatching  #natgeoyourshot #natureinfocus #indian_wildlifes #NaturePhotography #ThePhotoHour#wildlifephotography @Avibase"


api_url = "https://bird-name-ner-nlp.herokuapp.com/ner?sent="+sentence
response = requests.get(api_url).json()
print(response['bird-ner']) 
print(response['bird-wiki']) 
print(response['bird-ebird']) 
print(response['error']) 
print(response['messages']) 

['grey heron']
['grey heron']
[]
[]
['bird_list_df loaded', 'all birds loaded', '10975 birds list loaded.', 'user input recorded.', 'user input processed.']


# Prepping for Ensemble model


so currently there are two ways of finding the name: <br>
a. checking with the curated list of bird names.<br>
b. finding bird name using custom ner model. <br>

Let the two responses by X and Y. X: name we get using the curated list of bird names. and Y: name we get using the custom NER. <br>

Scenario 1: X && Y are same: Take X. <br>
Scenario 2: X has value && Y is blank: Take X. <br>
Scenario 3: X has value && Y is a substring of X: Take X. <br>
Scenario 4: X && Y are different: Take at least X. <br>
Scenario 5: X is blank && Y has value: Take common name of Y from eBird. <br>


In [ ]:
sentence = basic_preprocess(sentence,spelling_corrections)

In [ ]:
wikiBird_commonNames = all_birds["bird_name"].tolist()
core_dataset = pd.read_csv(google_drive_folder_path+"birdwatching"+'.csv', names=["created_at", "tweet_id", "user", "location", "tweet", "media_url", "hashtags"])
tweet_ids = core_dataset["tweet_id"].unique().tolist()
print(len(tweet_ids))

nlp_ner = spacy.load("model-best") 

def return_predicted_birdname(sentence):  
  results = {} #this is the result to the user.
  results['wiki'] = []
  results['eBird'] = []
  results['ner'] = []

  for birdname in wikiBird_commonNames: 
    if sentence.find(" "+birdname) > -1: results['wiki'].append(birdname) 

  for birdname in eBird_commonNames_list: 
    if sentence.find(" "+birdname) > -1: results['eBird'].append(birdname) 

  doc = nlp_ner(sentence)
  results['ner'] = doc.ents 

  return results

report = []
tweet_ids_done = []
for index, row in core_dataset.iterrows():
  if row["tweet_id"] not in tweet_ids_done: 
    sentence = basic_preprocess(row["tweet"],spelling_corrections)
    results = return_predicted_birdname(sentence)
    report.append([row["tweet_id"], sentence, results['wiki'], results['eBird'],results['ner']])
    tweet_ids_done.append(row["tweet_id"])

6136


In [ ]:
#report = pd.DataFrame(report, columns =['ID', 'tweet', 'Wiki', 'eBird', 'NER']) 

In [ ]:
from googlesearch import search 
def search_by_commonName_google(commonName): 
  results = search("eBird: "+commonName, tld="com", num=3, stop=3, pause=2) 
  for result in results: 
    if str(result).find("ebird.org") > -1:
      eBird_link = result
      length_ = len(eBird_link.split("/")) 
      speciesCode = eBird_link.split("/")[length_-1]
      return speciesCode 
  return ""

new_report = []
for row in report:
  dispute = False   
  if len(row[2]) ==0 and len(row[3]) ==0 and len(row[4]) !=0: 
    dispute = True  
    speciesCode = search_by_commonName_google(str(row[4][0]))
    if speciesCode in eBird_commonNames:  
      print(speciesCode, eBird_commonNames[speciesCode])
      if len(speciesCode)>1: 
        new_row = [row[0], row[1], row[2],row[3],row[4], eBird_commonNames[speciesCode]+"*"]
        print(row[2],row[3],row[4]) 
        print(new_row)
        new_report.append(new_row) 
        dispute = False 
    if dispute == True: 
      new_row = [row[0], row[1], row[2],row[3],row[4], "unresolved dispute"]
      print(row[2],row[3],row[4]) 
      print(new_row)
      new_report.append(new_row) 
      dispute = False 
  else: 
    new_report.append([row[0], row[1], row[2],row[3],row[4], "resolved dispute"]) 

In [ ]:
print(len(new_report), " total unique tweets.")
some_results = []
count_wiki = 0 
count_eBird = 0 
count_NER = 0
count_wiki_eBird_no_return = 0
count_wiki_eBird_both_return = 0
for line in new_report:
  if len(line[2]) == 0 and len(line[3]) == 0: 
    count_wiki_eBird_no_return += 1
  
  if len(line[2]) > 0 and len(line[3]) > 0: 
    count_wiki_eBird_both_return += 1

  if len(line[2]) > 0 or len(line[3]) > 0 or len(line[4]) > 0: 
    some_results.append([line[0],line[1],line[2],line[3],line[4],line[5]])

    if len(line[2])>0: count_wiki += 1 
    if len(line[3])>0: count_eBird += 1 
    if len(line[4])>0: count_NER += 1 
print(len(some_results), " had some prediction.")
print(count_wiki, " - predicted by Wikipedia.")
print(count_eBird, "  - predicted by eBird.")
print(count_NER, " - predicted by custom NER model.")
print(count_wiki_eBird_no_return, " - were not predicted either by Wiki or by eBird")
print(count_wiki_eBird_both_return, " - were predicted by Wiki AND by eBird")

6136  total unique tweets.
3752  had some prediction.
1823  - predicted by Wikipedia.
1938   - predicted by eBird.
3442  - predicted by custom NER model.
4134  - were not predicted either by Wiki or by eBird
1759  - were predicted by Wiki AND by eBird


In [ ]:
some_results_df = pd.DataFrame(some_results, columns =['id', 'tweet','Wiki', 'eBird', 'NER', 'resolution']) 

In [ ]:
some_results_df.to_csv(google_drive_folder_path+'revised_birdwatching_disputed results.csv')

# Reorganising the database to make search friendly.

In [ ]:
core_dataset = pd.read_csv(google_drive_folder_path+"indiAves"+'.csv', names=["created_at", "tweet_id", "user", "location", "tweet", "media_url", "hashtags"])

In [ ]:
import requests 
dataset_dict = {}
total_count = len(core_dataset)
print(total_count, "rows in dataset.")

count = 1
for index, row in core_dataset.iterrows(): 
  print("Processing", count, "/", total_count, "....")
  count += 1
  tweet = {} 
  tweet["date_created"] = row["created_at"]
  tweet["tweet_id"] = row["tweet_id"]
  tweet["tweet"] = row["tweet"] 
  tweet["user"] = row["user"]
  tweet["media_url"] = row["media_url"]
  tweet["hashtags"] = row["hashtags"]

  #find birds! 
  tweet_text = basic_preprocess(tweet["tweet"],spelling_corrections) 
  api_url = "https://bird-name-ner-nlp.herokuapp.com/ner?sent="+tweet_text
  response = requests.get(api_url).json()
  
  for bird in response['bird-wiki']: 
    isDone = False 
    if bird not in dataset_dict: 
      dataset_dict[bird] = []  
    
    for existing_tweet_ in dataset_dict[bird]: 
      if tweet["tweet_id"] == existing_tweet_["tweet_id"]: 
        isDone = True 
    
    if isDone == False: 
      dataset_dict[bird].append(tweet) 

  for bird in response['bird-ebird']: 
    isDone = False 
    if bird not in dataset_dict: 
      dataset_dict[bird] = []  
    
    for existing_tweet_ in dataset_dict[bird]: 
      if tweet["tweet_id"] == existing_tweet_["tweet_id"]: 
        isDone = True 
    
    if isDone == False: 
      dataset_dict[bird].append(tweet) 

  for bird in response['bird-ner']: 
    isDone = False 
    if bird not in dataset_dict: 
      dataset_dict[bird] = []  
    
    for existing_tweet_ in dataset_dict[bird]: 
      if tweet["tweet_id"] == existing_tweet_["tweet_id"]: 
        isDone = True 
    
    if isDone == False: 
      dataset_dict[bird].append(tweet)

In [ ]:
for key in dataset_dict:
  print(key, len(dataset_dict[key]))

In [ ]:
def export_complete_dataset(dataset_dict,path):
  filepath = open("/content/dataset_dictionary",'wb') 
  pickle.dump(dataset_dict, filepath)                     
  filepath.close()  

export_complete_dataset(complete_dataset_dict,google_drive_folder_path)

In [ ]:
dataset_dict['pied kingfisher']

In [ ]:
complete_dataset_dict = {} 
for bird in dataset_dict: 
  complete_dataset_dict[bird] = {} 
  complete_dataset_dict[bird]["count"] = len(dataset_dict[bird]) 
  complete_dataset_dict[bird]["entries"] = dataset_dict[bird] 
  complete_dataset_dict[bird]["eBird_link"] = "" 
  complete_dataset_dict[bird]["wiki_link"] = "" 
  complete_dataset_dict[bird]["botw_link"] = "" 
  complete_dataset_dict[bird]["ebird_description"] = "" 

In [ ]:
eBird_commonNames = get_eBird_commonNames_data(google_drive_folder_path)

In [ ]:
for bird in complete_dataset_dict: 
  commonName = bird
  for eBird in eBird_commonNames:
    if eBird_commonNames[eBird] == commonName: 
      complete_dataset_dict[bird]["eBird_link"] = "https://ebird.org/species/"+eBird
      complete_dataset_dict[bird]["botw_link"] = "https://birdsoftheworld.org/bow/species/"+eBird+"/cur/introduction"

In [ ]:
all_birds = load_all_birds_list(google_drive_folder_path)

In [ ]:
for bird in complete_dataset_dict: 
  for index, row in all_birds.iterrows():
    if bird == row["bird_name"]: 
      complete_dataset_dict[bird]["wiki_link"] = "https://en.wikipedia.org"+row["wiki_link"] 

In [ ]:
import urllib.request 
from bs4 import BeautifulSoup 
from urllib.request import urlopen  
import urllib.request
import html.parser 
from requests.exceptions import HTTPError
from socket import error as SocketError
from http.cookiejar import CookieJar
from urllib.request import build_opener, HTTPCookieProcessor

def return_html_code(url):
  opener = build_opener(HTTPCookieProcessor())
  response = opener.open(url)
  html = response.read() 
  return html

In [ ]:
def get_bird_description_from_ebird(eBird_link):
  soup = BeautifulSoup(return_html_code(eBird_link), 'html.parser')
  mat = soup.find_all("p", {"class": "u-stack-sm"})
  bird_description = mat[0].text
  return bird_description

In [ ]:
link = "https://ebird.org/species/colkin1"
print(get_bird_description_from_ebird(link))

A mid-sized kingfisher with a variable plumage pattern; nearly 50 subspecies described worldwide. All of them have a greenish-blue crown, a white spot above the lores, a black mask extending down as a narrow band across the hindneck, and a white collar. Upperparts are greenish-blue, rump brighter blue, with blue wings and tail and white underparts. Female and juvenile slightly duller. Subspecies mainly differ in plumage, with upperparts varying from blue to green to olive-toned, white loral spot sometimes extending as a supercilium, and underparts sometimes buffier. Gives carious vocalizations, but most common call is a “kek-kek-kek-kek.” Frequents mangroves and tidal creeks, though sometimes inhabits other coastal habitats, including coconut plantations in islands.


In [ ]:
complete_dataset_dict["collared kingfisher"]

for bird in complete_dataset_dict:
  if len(complete_dataset_dict[bird]['eBird_link'])>0: 
    try:
      complete_dataset_dict[bird]["ebird_description"] = get_bird_description_from_ebird(complete_dataset_dict[bird]['eBird_link'])
    except Exception as e:
      print(str(e), complete_dataset_dict[bird]['eBird_link'])


In [ ]:
complete_dataset_dict["collared kingfisher"]

In [ ]:
#export_complete_dataset(complete_dataset_dict,google_drive_folder_path)

In [ ]:
complete_dataset_dict['pied kingfisher']

In [ ]:
def get_complete_data(path):
  filepath = open(path+"dataset_dict",'rb') 
  complete_dataset_dict = pickle.load(filepath)
  filepath.close()  
  return complete_dataset_dict

complete_dataset_dict = get_complete_data(google_drive_folder_path)

In [ ]:
import requests
import shutil 
from PIL import Image
import pandas as pd 

In [ ]:
def crop_resize_image(filepath):
  im = Image.open(filepath)
  width, height = im.size
  if width > height: boom = height 
  else: boom = width 
  left = 0
  top = 0
  right = left + boom 
  bottom = top + boom 
  im1 = im.crop((left, top, right, bottom))
  newsize = (266, 266)
  im1 = im1.resize(newsize)

  try:
    im1.save(filepath)
    return True 
  except Exception as e:
    return False 

In [ ]:
def download_image(img_link):
  try:
    file_name=img_link.split("/")[len(img_link.split("/"))-1]
    file_name = google_drive_folder_path+"images/"+file_name
    res = requests.get(img_link, stream = True)
    if res.status_code == 200: 
      with open(file_name,'wb') as f:
          shutil.copyfileobj(res.raw, f)  
      crop_resize_image(file_name)
      return file_name    
    else: return "" 
  except Exception as e:
    print(str(e), img_link)

In [ ]:
complete_dataset_dict['eurasian wren']

{'count': 1,
 'entries': [{'date_created': '2022-09-10 14:05:15',
   'tweet_id': 1568601500087373824,
   'tweet': "b'The Eurasian wren or northern wren is a very small insectivorous bird, and the only member of the wren family Troglodytidae #IndiAves #BBCWildlifePOTD #birdwatching #ThePhotoHour #BirdsSeenIn2022 #NaturePhotograhpy @WildlifeMag @NatureattheBest #birds #birding @SonyBBCEarth https://t.co/YvOl1yag5g'",
   'user': "b'PranabMahanta0'",
   'media_url': 'http://pbs.twimg.com/media/FcTKvbdagAQ4sxk.jpg',
   'hashtags': "['IndiAves', 'BBCWildlifePOTD', 'birdwatching', 'ThePhotoHour', 'BirdsSeenIn2022', 'NaturePhotograhpy', 'birds', 'birding']"}],
 'eBird_link': 'https://ebird.org/species/winwre4',
 'wiki_link': 'https://en.wikipedia.org/wiki/Eurasian_wren',
 'botw_link': 'https://birdsoftheworld.org/bow/species/winwre4/cur/introduction',
 'ebird_description': 'Tiny brown bird that often holds its short little tail cocked up. Often sings from a fairly exposed perch, but at other t

In [ ]:
def get_all_image_links(complete_dataset_dict):
  bird_for_gallery = {} 
  for key in complete_dataset_dict: 
    download_link = "" 
    count = 0 
    while download_link == "":
      download_link = complete_dataset_dict[key]["entries"][count]["media_url"] 
      count += 1
      if count == len(complete_dataset_dict[key]["entries"]):
        break
    
    #if math.isnan(download_link) == False: 
    if type(download_link) == str: 
      file_name = download_image(download_link) 
      bird_for_gallery[key] = file_name
    else:
      print(math.isnan(download_link))

  return bird_for_gallery

In [ ]:
gallery_images = get_all_image_links(complete_dataset_dict) 

In [ ]:
gallery_images

{'pied kingfisher': '/content/drive/My Drive/IndiAves/images/FbLq_JsagAI5Mt1.jpg',
 'coopers hawk': '/content/drive/My Drive/IndiAves/images/FbLjTiNaIAA5Sp3.jpg',
 'indian pitta': '/content/drive/My Drive/IndiAves/images/FbLiCpqaAAsRPTW.jpg',
 'collared kingfisher': '/content/drive/My Drive/IndiAves/images/FbLaq9yUYAAKvGK.jpg',
 'black naped tern': '/content/drive/My Drive/IndiAves/images/FbLaZFDUUAE68eh.jpg',
 'red breasted flycatcher': '/content/drive/My Drive/IndiAves/images/FbLZaOaaIAIU8FZ.jpg',
 'little blue heron': '/content/drive/My Drive/IndiAves/images/FbLKAwPUUAQr9Iu.jpg',
 'gray hornbill': '/content/drive/My Drive/IndiAves/images/FbLJzY0VEAMGbkf.jpg',
 'white rumped shams': '/content/drive/My Drive/IndiAves/images/FbLJZjkVEAAyfEr.jpg',
 'yellow wattled lapwing': '/content/drive/My Drive/IndiAves/images/FbK4N_tacAAG-_0.jpg',
 'wattled lapwing': '/content/drive/My Drive/IndiAves/images/FbK4N_tacAAG-_0.jpg',
 'red backed woodpecker': '/content/drive/My Drive/IndiAves/images/FbK

In [ ]:
bird_gallery

# Google connection for saving images.

In [ ]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [ ]:
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError